[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/CO-CONNECT/co-connect-tools/HEAD)


## Installing

The best way is to install the module via `pip`. 

In [ ]:
!pip3 install co-connect-tools -q

In [1]:
import coconnect.tools
data = coconnect.tools.load_json('example/sample_config/lion_structural_mapping.json')
coconnect.tools.extract.make_class(data,'Lion')

Recreating file /Users/calummacdonald/Usher/CO-CONNECT/Software/docs/docs/co-connect-tools/coconnect/cdm/classes/Lion.py


In [2]:
print (coconnect.tools.get_classes(format=True))

{
      "Lion": {
            "module": "coconnect.cdm.classes.Lion",
            "path": "/Users/calummacdonald/Usher/CO-CONNECT/Software/docs/docs/co-connect-tools/coconnect/cdm/classes/Lion.py",
            "last-modified": "2021-03-31 12:44:01"
      }
}


In [3]:
f_map = coconnect.tools.get_file_map_from_dir('example/sample_input_data/')
f_map

{'track.csv': '/Users/calummacdonald/Usher/CO-CONNECT/Software/docs/docs/co-connect-tools/coconnect/data/example/sample_input_data/track.csv',
 'demo.csv': '/Users/calummacdonald/Usher/CO-CONNECT/Software/docs/docs/co-connect-tools/coconnect/data/example/sample_input_data/demo.csv'}

In [4]:
inputs = coconnect.tools.load_csv(f_map)

In [5]:
from coconnect.cdm.classes.Lion import Lion
lion = Lion(inputs=inputs)

2021-03-31 12:44:01 - Lion - INFO - CommonDataModel created


In [6]:
lion.process()

2021-03-31 12:44:01 - Lion - INFO - for person: found 1 object
2021-03-31 12:44:01 - Lion - INFO - working on <class 'coconnect.cdm.objects.person.Person'>
2021-03-31 12:44:01 - Lion - INFO - finished person_0 ... 0/1, 4 rows
2021-03-31 12:44:01 - Lion - INFO - Merging 1 objects for <class 'coconnect.cdm.objects.person.Person'>
2021-03-31 12:44:01 - Lion - INFO - Finalising <class 'coconnect.cdm.objects.person.Person'>
2021-03-31 12:44:01 - person_0 - INFO - Just masked person_id
2021-03-31 12:44:01 - Lion - INFO - Formating the output for <class 'coconnect.cdm.objects.person.Person'>
2021-03-31 12:44:01 - Lion - INFO - finalised person
2021-03-31 12:44:01 - Lion - INFO - for condition_occurrence: found 0 object
2021-03-31 12:44:01 - Lion - INFO - finalised condition_occurrence
2021-03-31 12:44:01 - Lion - INFO - for visit_occurrence: found 0 object
2021-03-31 12:44:01 - Lion - INFO - finalised visit_occurrence
2021-03-31 12:44:01 - Lion - INFO - for measurement: found 0 object
2021-03

In [7]:
lion.omop['person']

,gender_concept_id,year_of_birth,month_of_birth,day_of_birth,birth_datetime,race_concept_id,ethnicity_concept_id,location_id,provider_id,care_site_id,person_source_value,gender_source_value,gender_source_concept_id,race_source_value,race_source_concept_id,ethnicity_source_value,ethnicity_source_concept_id
person_id,,,,,,,,,,,,,,,,,
1,8532,1962,7,18,1962-07-18 00:00:00,4196428,<NA>,<NA>,<NA>,<NA>,,F,8532,WHITE BRITISH,4196428,,<NA>
2,8532,1973,7,20,1973-07-20 00:00:00,4196428,<NA>,<NA>,<NA>,<NA>,,F,8532,WHITE BRITISH,4196428,,<NA>
3,8532,1959,7,15,1959-07-15 00:00:00,4087917,<NA>,<NA>,<NA>,<NA>,,F,8532,BLACK CARIBBEAN,4087917,,<NA>
4,8532,1982,3,7,1982-03-07 00:00:00,4196428,<NA>,<NA>,<NA>,<NA>,,F,8532,WHITE BRITISH,4196428,,<NA>


## Load Inputs

To run the tool you need to load some input datasets, and specify how to map the fields 

The data will be loaded into pandas dataframes that we'll use for some visualisations of what the input `csv` files will look like

### Source data

This data is synthetic data that has been produced by [OHDSI](http://ohdsi.org/) which simply details a record of patients.

_Note: that these example data files will be stored in `<install_dir>/lib/python3.8/site-packages/coconnect/`, a directory that `ETLTool` will be looking in. For your own files, you should specify the full path to the inputs_

In [ ]:
f_input_data = 'sample_input_data/patients_sample.csv'
etltool.load_input_data([f_input_data])

Verify what files have been loaded, by default the input dataset is mapped to to a name via `/path/<name>.csv`

In [ ]:
etltool.get_input_names()

Sample (3 entries) what this input data looks like. __Note__ becareful using this method with a large dataset

In [ ]:
df_input = etltool.get_input_df('patients_sample.csv')
df_input.sample(3)

### Structural Mapping

Next we use another `csv` file to define how to map different fields in the source data to a [Common Data Model (CDM)](https://www.ohdsi.org/data-standardization/the-common-data-model/).

In this example, the CDM that the source data (`patients_sample`)  is being mappped to is the `person` CDM.

There are three rules defined:
1. Performs a straight one-to-one mapping between the field `id` in the source field to the `person_id` field of the `person` CDM
2. Performs a mapping with the operation/function `extract year` 
3. Performs a term mapping which is defined in the term mapping `csv` file, see the next section for more information 


In [ ]:
f_structural_mapping = 'sample_input_data/rules1.csv'
etltool.load_structural_mapping(f_structural_mapping) 
etltool.get_structural_mapping_df()

#### Testing operations
The 2nd rule defined the operation `extract year`, this is a default operation defined in `etltool`, here is a quick example of how it works..

Load the function

In [ ]:
fn_extract_year = etltool.allowed_operations['EXTRACT_YEAR']
fn_extract_year

For example, taking the `BIRTHDATE` columns, which looks like:

In [ ]:
df_input['BIRTHDATE'].head(4)

The function can be used to easily extract the year from the date

In [ ]:
fn_extract_year(df_input.head(4),column='BIRTHDATE')

### Term Mapping

In the term mapping, the structural mapping `rule_id` is mapped telling us how to map a source term to a destination term, i.e. if the source term is `M` then the output should be `8507`

In [ ]:
f_term_mapping = 'sample_input_data/rules2.csv'
etltool.load_term_mapping(f_term_mapping)
etltool.get_term_mapping_df()

### Run the tool

In [ ]:
etltool.run()

We can finally get the output in a dataframe

In [ ]:
etltool.get_output_df('person')